In [1]:
import pandas as pd
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from node2vec import Node2Vec
from time import time

In [2]:
train = pd.read_csv("data_train_edge.csv")
predict = pd.read_csv("predict.csv")
print("training pairs： %s, predict pairs： %s" % (len(train),len(predict)))

training pairs： 20457, predict pairs： 10231


In [3]:
train.head(5)

,node1,node2
0,287,68
1,63,552
2,189,20
3,380,376
4,370,443


# origin train graph

In [4]:
# create graph
G = nx.from_pandas_edgelist(train, "node1", "node2", create_using=nx.DiGraph())

In [5]:
n = G.number_of_nodes()
m = G.number_of_edges()
print("Number of nodes :", str(n))
print("Number of edges :", str(m))

Number of nodes : 979
Number of edges : 20457


In [6]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 979
Number of edges: 20457
Average in degree:  20.8958
Average out degree:  20.8958


# generate missing edges

In [7]:
#the dict will contain a tuple of 2 nodes as key and the value will be 1 if the nodes are connected
G = nx.from_pandas_edgelist(train, "node1", "node2", create_using=nx.DiGraph())
edges = dict()
for edge in train.values:
    edges[(edge[0], edge[1])] = 1
    
missing_edges = set([])
while (len(missing_edges)< G.number_of_edges()):
    a=random.randint(0, 1004)
    b=random.randint(0, 1004)
    tmp = edges.get((a,b),-1) # if key(a,b) is not in edges(connected nodes) then return -1, otherwise return 1 
    if tmp == -1 and a!=b:
        try:
            # adding points who less likely to be friends
            if nx.shortest_path_length(G,source=a,target=b) > 2: 
                missing_edges.add((a,b))
            else:
                continue  
        except:  
            missing_edges.add((a,b))              
    else:
        continue

In [8]:
print('# of positive samples: %d, # of negative samples: %d' % (len(edges),len(missing_edges)))

# of positive samples: 20457, # of negative samples: 20457


In [9]:
pos = pd.DataFrame(list(edges), columns=['node1', 'node2'])
neg = pd.DataFrame(list(missing_edges), columns=['node1', 'node2'])

In [10]:
train_all = pos.append(neg,ignore_index=True)
y_train_all = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))

# train-test split

In [11]:
# total data 
df_pos = pd.read_csv('data_train_edge.csv')
df_neg = pd.DataFrame(list(missing_edges), columns=['node1', 'node2'])
print("Number of nodes in the graph with edges", df_pos.shape[0])
print("Number of nodes in the graph without edges", df_neg.shape[0])
 
# Spilt data into 70-30 positive/negative links seperatly,we only need "positive training data" for creating graph and features
X_train_pos, X_test_pos, y_train_pos, y_test_pos  = train_test_split(df_pos,np.ones(len(df_pos)),test_size=0.3, random_state=9)
X_train_neg, X_test_neg, y_train_neg, y_test_neg  = train_test_split(df_neg,np.zeros(len(df_neg)),test_size=0.3, random_state=9)
print('\n')
print("Number of nodes in the train data graph with edges", X_train_pos.shape[0])
print("Number of nodes in the train data graph without edges", X_train_neg.shape[0])
print('\n')
print("Number of nodes in the test data graph with edges", X_test_pos.shape[0])
print("Number of nodes in the test data graph without edges", X_test_neg.shape[0])

Number of nodes in the graph with edges 20457
Number of nodes in the graph without edges 20457


Number of nodes in the train data graph with edges 14319
Number of nodes in the train data graph without edges 14319


Number of nodes in the test data graph with edges 6138
Number of nodes in the test data graph without edges 6138


In [12]:
train_graph=nx.from_pandas_edgelist(X_train_pos, "node1", "node2", create_using=nx.DiGraph())
test_graph=nx.from_pandas_edgelist(X_test_pos, "node1", "node2", create_using=nx.DiGraph())
print(nx.info(train_graph),'\n')
print(nx.info(test_graph),'\n')

# find the unique nodes in the both train and test graphs
train_nodes_pos = set(train_graph.nodes())
test_nodes_pos = set(test_graph.nodes())

trY_teY = len(train_nodes_pos.intersection(test_nodes_pos))
trY_teN = len(train_nodes_pos - test_nodes_pos)
teY_trN = len(test_nodes_pos - train_nodes_pos)

print('# of people common in train and test -- ',trY_teY)
print('# of people present in train but not present in test -- ',trY_teN)
print('# of people present in test but not present in train -- ',teY_trN) #比較沒有冷啟動問題
print(' % of people not in Train but exist in Test in total Test data are {} %'.format(teY_trN/len(test_nodes_pos)*100))

Name: 
Type: DiGraph
Number of nodes: 941
Number of edges: 14319
Average in degree:  15.2168
Average out degree:  15.2168 

Name: 
Type: DiGraph
Number of nodes: 881
Number of edges: 6138
Average in degree:   6.9671
Average out degree:   6.9671 

# of people common in train and test --  843
# of people present in train but not present in test --  98
# of people present in test but not present in train --  38
 % of people not in Train but exist in Test in total Test data are 4.313280363223609 %


In [13]:
# concatenate positive & negative samples
X_train = X_train_pos.append(X_train_neg,ignore_index=True)
y_train = np.concatenate((y_train_pos,y_train_neg))
X_test = X_test_pos.append(X_test_neg,ignore_index=True)
y_test = np.concatenate((y_test_pos,y_test_neg)) 

In [14]:
print("X_train: %s, X_test: %s " % (len(X_train), len(X_test)))

X_train: 28638, X_test: 12276 


# train-validation split

In [15]:
X_train_tra_pos, X_train_val_pos, y_train_tra_pos, y_train_val_pos  = train_test_split(X_train_pos,np.ones(len(X_train_pos)),test_size=0.2, random_state=9)
X_train_tra_neg, X_train_val_neg, y_train_tra_neg, y_train_val_neg  = train_test_split(X_train_neg,np.zeros(len(X_train_neg)),test_size=0.2, random_state=9)

In [16]:
train_tra_graph=nx.from_pandas_edgelist(X_train_tra_pos, "node1", "node2", create_using=nx.DiGraph())
train_val_graph=nx.from_pandas_edgelist(X_train_val_pos, "node1", "node2", create_using=nx.DiGraph())
print(nx.info(train_tra_graph),'\n')
print(nx.info(train_val_graph),'\n')

Name: 
Type: DiGraph
Number of nodes: 917
Number of edges: 11455
Average in degree:  12.4918
Average out degree:  12.4918 

Name: 
Type: DiGraph
Number of nodes: 778
Number of edges: 2864
Average in degree:   3.6812
Average out degree:   3.6812 



In [17]:
# concatenate positive & negative samples
X_train_tra = X_train_tra_pos.append(X_train_tra_neg, ignore_index=True)
y_train_tra = np.concatenate((y_train_tra_pos, y_train_tra_neg))
X_train_val = X_train_val_pos.append(X_train_val_neg, ignore_index=True)
y_train_val = np.concatenate((y_train_val_pos, y_train_val_neg)) 

In [18]:
print("X_train_train: %s, X_train_validation: %s " % (len(X_train_tra), len(X_train_val)))

X_train_train: 22910, X_train_validation: 5728 


# Features(proximity score)

In [19]:
'''
A "successor" of n is a node m such that there exists a directed edge "from n to m" (n->m ,n向外指向的nodes數)
A "predecessor" of n is a node m such that there exists a directed edge "from m to n" (m->n ,指向n的nodes數)
此處因是directed graph，定義「被指向者」為朋友，ex：a->b，b為a的朋友
'''
# 1.Jaccard_coefficient
def cal_jaccard_coefficient(a,b,train_graph):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                 (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
        return sim
    except:
        return 0

# 2.Adar Index
def calc_adar_in(a,b,train_graph):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))) #同時被 a、b指向者(common friends)
        if len(n)!=0:
            for i in n:
                a = len(list(train_graph.predecessors(i))) #每一個common friend的朋友數量(指向 i 的數量)
                if a > 1:
                    sum =sum +(1/np.log10(a)) # i 的朋友數越多，權重佔比越小 
                else:
                    continue
            return sum
        else:
            return 0
    except:
        return 0

In [20]:
# 3.Shortest path
def compute_shortest_path_length(a,b,train_graph):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

In [21]:
# 4.Follow back
def follows_back(a,b,train_graph):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

In [22]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 979
Number of edges: 20457
Average in degree:  20.8958
Average out degree:  20.8958


In [23]:
print(nx.info(train_graph))

Name: 
Type: DiGraph
Number of nodes: 941
Number of edges: 14319
Average in degree:  15.2168
Average out degree:  15.2168


In [24]:
print(nx.info(train_tra_graph))

Name: 
Type: DiGraph
Number of nodes: 917
Number of edges: 11455
Average in degree:  12.4918
Average out degree:  12.4918


In [25]:
# 5.Page rank(count the number and quality of nodes to determine how important the node is)
'''
For the data points which exists in test dataset but not exists in the train dataset will not have pagerank value.
For these data points,just use the mean pagerank as imputation.
'''
def pg_rank(train_graph):
    pr = nx.pagerank(train_graph, alpha=0.85) #dict
    mean_pr = float(sum(pr.values())) / len(pr)
    print("pg rank")
    print('min',pr[min(pr, key=pr.get)])
    print('max',pr[max(pr, key=pr.get)])
    print('mean',mean_pr)
    return pr, mean_pr

In [26]:
# 6.Katz Centrality(measure influence by taking into account the total number of walks between a pair of nodes)
def katz_central(train_graph):
    katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    mean_katz = float(sum(katz.values())) / len(katz)
    print('Katz Centrality')
    print('min',katz[min(katz, key=katz.get)])
    print('max',katz[max(katz, key=katz.get)])
    print('mean',mean_katz)
    return katz, mean_katz

In [27]:
# 7.HITS
'''
HITS identifies good authorities and hubs for a topic by assigning two numbers to a node : an authority and a hub weight. 
Authorities estimate the node value based on the incoming links. 
Hubs estimates the node value based on outgoing links.
'''
def HITS(train_graph):
    hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    mean_hits = float(sum(hits[0].values())) / len(hits[0])
    print('Hyper-link induced topic search (HITS)')
    print('min',hits[0][min(hits[0], key=hits[0].get)])
    print('max',hits[0][max(hits[0], key=hits[0].get)])
    print('mean',mean_hits,'\n')
    return hits, mean_hits

In [28]:
def compute_features_follow(df_final):
    #calculating no of followers followees for node1 and node2
    #calculating intersection of followers and followees for node1 and node2
    num_followers_1=[]
    num_followees_1=[]
    num_followers_2=[]
    num_followees_2=[]
    inter_followers=[]
    inter_followees=[]
    for i,row in df_final.iterrows():
        try:
            n1_p=set(train_graph.predecessors(row['node1']))
            n1_s=set(train_graph.successors(row['node1']))
        except:
            n1_p = set()
            n1_s = set()
        try:
            n2_p =set(train_graph.predecessors(row['node2']))
            n2_s =set(train_graph.successors(row['node2']))
        except:
            n2_p = set()
            n2_s = set()
        num_followers_1.append(len(n1_p))
        num_followees_1.append(len(n1_s))

        num_followers_2.append(len(n2_p))
        num_followees_2.append(len(n2_s))

        inter_followers.append(len(n1_p.intersection(n2_p)))
        inter_followees.append(len(n1_s.intersection(n2_s)))
    
    return num_followers_1, num_followers_2, num_followees_1, num_followees_2, inter_followers, inter_followees

# Node2Vec

In [29]:
# Precompute probabilities and generate walks 
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=10)  

# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4) 

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.41it/s]


In [30]:
def cosin_distance(vector1, vector2):
    dot_product = 0.0
    normA = 0.0
    normB = 0.0
    for a, b in zip(vector1, vector2):
        dot_product += a * b
        normA += a ** 2
        normB += b ** 2
    if normA == 0.0 or normB == 0.0:
        return None
    else:
        return dot_product / ((normA * normB) ** 0.5)

def cos_sim(x_train):
    node_sim = [] 
    for i,j in x_train[['node1','node2']].values:
        try:
            sim = cosin_distance(model.wv.get_vector(str(i)),model.wv.get_vector(str(j)))
        except:
            sim = -1
        node_sim.append(sim)
    
    final_sim = np.round(node_sim,3)
    return final_sim

# map features to data

In [31]:
def map_features(X_train, train_graph, df_to_write_in, version, link_y):
    start_time = time()

    X_train['jaccard_coef'] = X_train.apply(lambda row:cal_jaccard_coefficient(row['node1'],row['node2'],train_graph),axis=1)
    X_train['adar_index'] = X_train.apply(lambda row: calc_adar_in(row['node1'],row['node2'],train_graph),axis=1)
    X_train['shortest_path'] = X_train.apply(lambda row: compute_shortest_path_length(row['node1'],row['node2'],train_graph),axis=1)
    X_train['follows_back'] = X_train.apply(lambda row: follows_back(row['node1'],row['node2'],train_graph),axis=1)
    
    pr, mean_pr = pg_rank(train_graph)
    X_train['page_rank_n1'] = X_train.node1.apply(lambda x:pr.get(x,mean_pr))
    X_train['page_rank_n2'] = X_train.node2.apply(lambda x:pr.get(x,mean_pr))

    katz, mean_katz = katz_central(train_graph)
    X_train['katz_n1'] = X_train.node1.apply(lambda x: katz.get(x,mean_katz))
    X_train['katz_n2'] = X_train.node2.apply(lambda x: katz.get(x,mean_katz))

    hits, mean_hits = HITS(train_graph)
    X_train['hubs_n1'] = X_train.node1.apply(lambda x: hits[0].get(x,0))
    X_train['hubs_n2'] = X_train.node2.apply(lambda x: hits[0].get(x,0))
    X_train['authorities_n1'] = X_train.node1.apply(lambda x: hits[1].get(x,0))
    X_train['authorities_n2'] = X_train.node2.apply(lambda x: hits[1].get(x,0))

    X_train['num_followers_node1'], X_train['num_followers_node2'], \
    X_train['num_followees_node1'], X_train['num_followees_node2'], \
    X_train['inter_followers'], X_train['inter_followees']= compute_features_follow(X_train)
    
    X_train['cos_sim'] = cos_sim(X_train)
    
    X_train['link'] = link_y
    print("--- %s seconds ---" % (time() - start_time))
    X_train.to_csv('features/%s_%d.csv'% (df_to_write_in, version), index=False)
    
    return X_train

In [32]:
x_train_train = map_features(X_train_tra, train_tra_graph, 'X_train_train', 10, y_train_tra)

pg rank
min 0.000232166384117935
max 0.006136821049612314
mean 0.0010905125408942201
Katz Centrality
min 0.030762336873721593
max 0.046258088955172603
mean 0.03294528736512956
Hyper-link induced topic search (HITS)
min 0.0
max 0.010698703398751114
mean 0.0010905125408942203 

--- 11.99114179611206 seconds ---


In [33]:
x_train_valid = map_features(X_train_val, train_tra_graph, 'X_train_valid', 10, y_train_val)

pg rank
min 0.000232166384117935
max 0.006136821049612314
mean 0.0010905125408942201
Katz Centrality
min 0.030762336873721593
max 0.046258088955172603
mean 0.03294528736512956
Hyper-link induced topic search (HITS)
min 0.0
max 0.010698703398751114
mean 0.0010905125408942203 

--- 4.356195449829102 seconds ---


In [34]:
x_train_train.head(2)

,node1,node2,jaccard_coef,adar_index,shortest_path,follows_back,page_rank_n1,page_rank_n2,katz_n1,katz_n2,...,authorities_n1,authorities_n2,num_followers_node1,num_followers_node2,num_followees_node1,num_followees_node2,inter_followers,inter_followees,cos_sim,link
0,377,464,0.015152,0.642549,2,0,0.002488,0.001479,0.037038,0.033977,...,0.002204,0.001710,44,24,75,16,2,1,0.196,1.0
1,918,974,0.000000,0.000000,3,1,0.000745,0.000490,0.032169,0.031289,...,0.000527,0.000206,10,5,7,5,1,2,0.950,1.0


In [35]:
X_train = map_features(X_train, train_graph, 'X_train', 10, y_train)

pg rank
min 0.00021927129612718288
max 0.017357880091321273
mean 0.0010626992561105241
Katz Centrality
min 0.029790433793679696
max 0.04851136786130019
mean 0.032474922437919156
Hyper-link induced topic search (HITS)
min 0.0
max 0.011596257114007726
mean 0.001062699256110521 

--- 15.936713695526123 seconds ---


In [39]:
X_train.describe()

,node1,node2,jaccard_coef,adar_index,shortest_path,follows_back,page_rank_n1,page_rank_n2,katz_n1,katz_n2,...,authorities_n1,authorities_n2,num_followers_node1,num_followers_node2,num_followees_node1,num_followees_node2,inter_followers,inter_followees,cos_sim,link
count,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,...,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000,28638.000000
mean,433.440603,434.674733,0.053925,1.973485,1.387667,0.206195,0.001383,0.001394,0.033734,0.033659,...,0.001550,0.001526,21.367519,20.996787,23.973357,21.586563,2.879740,2.852713,0.339283,0.500000
std,289.940688,287.791149,0.121216,3.692951,1.651564,0.404579,0.001160,0.001305,0.003794,0.003684,...,0.001727,0.001694,21.574578,20.880349,28.250860,25.549789,4.796541,5.197039,0.433485,0.500009
min,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000219,0.000219,0.029790,0.029790,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
25%,176.000000,182.000000,0.000000,0.000000,-1.000000,0.000000,0.000605,0.000618,0.030958,0.030978,...,0.000258,0.000265,5.000000,5.000000,4.000000,3.000000,0.000000,0.000000,0.079000,0.000000
50%,401.000000,402.000000,0.000000,0.000000,2.000000,0.000000,0.001063,0.001063,0.032506,0.032504,...,0.000895,0.000897,15.000000,15.000000,15.000000,14.000000,0.000000,0.000000,0.328000,0.500000
75%,671.000000,671.000000,0.074074,2.951680,3.000000,0.000000,0.001767,0.001733,0.035155,0.035017,...,0.002319,0.002201,30.000000,29.000000,34.000000,30.000000,5.000000,4.000000,0.698000,1.000000
max,1004.000000,1004.000000,1.000000,93.290464,6.000000,1.000000,0.017358,0.017358,0.048511,0.048511,...,0.007640,0.007640,106.000000,106.000000,181.000000,181.000000,105.000000,123.000000,1.000000,1.000000


In [36]:
X_test = map_features(X_test, train_graph, 'X_test', 10, y_test)

pg rank
min 0.00021927129612718288
max 0.017357880091321273
mean 0.0010626992561105241
Katz Centrality
min 0.029790433793679696
max 0.04851136786130019
mean 0.032474922437919156
Hyper-link induced topic search (HITS)
min 0.0
max 0.011596257114007726
mean 0.001062699256110521 

--- 8.745077133178711 seconds ---


In [37]:
train_all = map_features(train_all, G, 'train_all', 10, y_train_all)

pg rank
min 0.00019950526574374428
max 0.018093180173678888
mean 0.0010214504596527093
Katz Centrality
min 0.02788899829926249
max 0.057905326386675705
mean 0.03168690897115153
Hyper-link induced topic search (HITS)
min 0.0
max 0.011219812058962837
mean 0.001021450459652707 

--- 27.365792751312256 seconds ---


In [38]:
predict = map_features(predict, G, 'predict_all', 10, np.zeros(len(predict)))

pg rank
min 0.00019950526574374428
max 0.018093180173678888
mean 0.0010214504596527093
Katz Centrality
min 0.02788899829926249
max 0.057905326386675705
mean 0.03168690897115153
Hyper-link induced topic search (HITS)
min 0.0
max 0.011219812058962837
mean 0.001021450459652707 

--- 9.689629077911377 seconds ---
